In [1]:
%pip install dashscope faiss-cpu sentence-transformers

Looking in indexes: https://mirrors.tencent.com/pypi/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 9.1 MB/s eta 0:00:0000:01:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 8.0 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.5/223.5 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.5/232.5 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.1/358.1 kB 9.4 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from dashscope import Generation
import re

# ====== 配置部分 ======
FAISS_INDEX_PATH = 'legal_clauses_faiss.index'  # FAISS索引文件路径
MODEL_NAME = 'paraphrase-multilingual-MiniLM-L12-v2'  # 语义模型
QWEN_API_KEY = 'sk-7fb2aee47f5d4531855a7ac3412249fe'  # 替换为你的DashScope API Key
SEARCH_TOP_K = 3  # 每次检索返回的最相关条款数量

# ====== 系统初始化 ======
# 加载语义模型
model = SentenceTransformer(MODEL_NAME)
# 加载FAISS索引
index = faiss.read_index(FAISS_INDEX_PATH)

# ====== 核心功能函数 ======
def legal_judgment_system(user_input):
    # 步骤1：检索相关法律条款
    query_embedding = model.encode([user_input])
    distances, indices = index.search(query_embedding.astype(np.float32), SEARCH_TOP_K)
    
    # 读取法律条款库
    with open('laws.txt', 'r', encoding='utf-8') as f:
        all_clauses = f.read().split('\n')
    
    # 获取相关条款文本
    relevant_clauses = [all_clauses[idx] for idx in indices[0] if idx < len(all_clauses)]
    
    # 步骤2：构建Prompt
    prompt_template = """
    [任务说明]
    你是一个法律合规性审查助手，需要根据提供的法律条款对用户输入的政策文本进行合规性判断。

    [相关法律条款]
    {clauses}

    [待审查政策]
    {input}

    [输出要求]
    1. 用一句话给出合规性判断（合规/部分合规/不合规）
    2. 简要说明理由（50字以内）
    3. 引用具体法律条款（条款编号）
    请用以下格式输出：
    | 判断结果 | 理由 | 法律依据 |
    """
    
    prompt = prompt_template.format(
        clauses='\n'.join(relevant_clauses),
        input=user_input
    )

    # 步骤3：调用Qwen模型
    response = Generation.call(
        model='qwen-turbo',
        prompt=prompt,
        api_key=QWEN_API_KEY,
        max_length=500,
        top_p=0.8
    )
    
    # 解析输出
    if response.status_code == 200:
        return response.output.text
    else:
        return f"错误：{response.message}"

# ====== 使用示例 ======
if __name__ == "__main__":
    test_input = "地方政府要求所有网约车平台必须使用指定公司的安全监测系统"
    result = legal_judgment_system(test_input)
    
    # 格式化输出
    print("=== 输入政策 ===")
    print(test_input)
    print("\n=== 合规性判断 ===")
    print(result)
    
    # 提取结构化结果（可选）
    # if '|' in result:
    #     parts = [p.strip() for p in result.split('|') if p.strip()]
    #     if len(parts) >= 3:
    #         print(f"\n结构化结果：\n- 合规性：{parts[0]}\n- 理由：{parts[1]}\n- 依据：{parts[2]}")


=== 输入政策 ===
地方政府要求所有网约车平台必须使用指定公司的安全监测系统

=== 合规性判断 ===
| 判断结果 | 不合规 | 理由：排斥、限制外地经营者参加本地采购。 | 第三条（三） |

结构化结果：
- 合规性：判断结果
- 理由：不合规
- 依据：理由：排斥、限制外地经营者参加本地采购。
